# **Collecte des Données via l'API**

Pour effectuer la collecte des données, nous sommes allé chercher le lien sur **edamam** en creant notre compte ensuite creer une aplication afin d'avoir app_id et app_key. À partir de ces données, nous allons sélectionner les colonnes demandées et intégrer ces informations dans un fichier CSV.

Enfin, nous vérifierons que les 10 premières lignes sont correctement récupérées.

In [ ]:
pip install numpy pandas scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msn
import seaborn as sns

In [ ]:
# Importer les bibliothèques nécessaires
from google.colab import drive
from PIL import Image
import os
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
import cv2
import time
from sklearn import manifold, decomposition
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [ ]:
import requests

# URL de l'API pour la recherche d'aliments
url = "https://api.edamam.com/api/food-database/v2/parser"

# Paramètres de la requête
querystring = {
    "app_id": "45090373",  # Mon ID d'application
    "app_key": "a90a30519bc635a48bc2a6b050d50670",  # Ma clé d'application
}

# Liste d'ingrédients à tester
ingredients = [
    "pomme",
    "banane",
    "avocat",
    "poulet",
    "brocoli",
    "quinoa",
    "thon",
    "champignon",
    "carotte",
    "chocolat"
]

# Parcourir la liste des ingrédients
for ingredient in ingredients:
    querystring["ingr"] = ingredient  # Mettre à jour l'ingrédient
    response = requests.get(url, params=querystring)  # Envoyer la requête
    json_data = response.json()  # Convertir la réponse en JSON
    print(f"Résultats pour {ingredient} : {json_data}")


Résultats pour pomme : {'text': 'pomme', 'count': 14, 'parsed': [], 'hints': [{'food': {'foodId': 'food_bv970bwb6ss4kdbavmuqoaeqiyjf', 'label': 'Pommes Dauphinoise', 'knownAs': 'Pommes Dauphinoise', 'nutrients': {'ENERC_KCAL': 176.49338749701462, 'PROCNT': 5.030143880204679, 'FAT': 12.642582763706834, 'CHOCDF': 11.440705703072945, 'FIBTG': 1.3290334811780267}, 'category': 'Generic meals', 'categoryLabel': 'meal', 'foodContentsLabel': 'butter; heavy cream; milk; garlic; salt; white pepper; Yukon Gold potatoes; gruyere cheese; fresh rosemary'}, 'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_unit', 'label': 'Whole', 'weight': 352.0347239490349}, {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_serving', 'label': 'Serving', 'weight': 352.0347239490349}, {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_gram', 'label': 'Gram', 'weight': 1.0}, {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_ounce', 'label': 'Ounce', 'weight': 28.

In [ ]:
import requests
import json
import pandas as pd
import csv

# URL de l'API pour obtenir les données
url = "https://api.edamam.com/api/food-database/v2/parser"

# Paramètres pour l'API, dans ce cas pour chercher des informations sur le champagne
querystring = {
    "app_id": "45090373",
    "app_key": "a90a30519bc635a48bc2a6b050d50670",
    "ingr": "champagne"  # L'aliment recherchés
}

# Envoyer la requête pour l'API et obtenir la réponse
response = requests.get(url, params=querystring)

# Charger la réponse en format JSON
json_data = response.json()

# Extraire les données pertinentes et les structurer en une liste de dictionnaires
csv_data = []
for item in json_data.get('hints', []):  # Utilisation de get pour éviter une KeyError
    row = {
        'foodId': item['food']['foodId'],
        'label': item['food']['label'],
        'category': item['food']['category'],
        'foodContentsLabel': item['food'].get('foodContentsLabel'),
        'image': item['food'].get('image')
    }
    csv_data.append(row)

**creer le fichier en csv**

In [ ]:
# Écrire les données CSV dans un fichier
with open('output.csv', 'w', newline='') as csv_file:
    fieldnames = ['foodId', 'label', 'category', 'foodContentsLabel', 'image']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()  # Écriture de l'en-tête
    for row in csv_data:
        writer.writerow(row)  # Écriture de chaque ligne

# Charger les données CSV dans un DataFrame pandas
data_food = pd.read_csv('output.csv', nrows=10, usecols=['foodId', 'label', 'category', 'foodContentsLabel', 'image'])

# Exporter le DataFrame en fichier CSV final
data_food.to_csv('output_final.csv', index=False)

# Vérification du nombre de lignes et colonnes
shape = data_food.shape
print(f"Le DataFrame contient {shape[0]} lignes et {shape[1]} colonnes.")

Le DataFrame contient 10 lignes et 5 colonnes.


In [ ]:
data_food

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
8,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN
9,food_a79xmnya6togreaeukbroa0thhh0,Champagne Chicken,Generic meals,"Flour; Salt; Pepper; Boneless, Skinless Chicke...",NaN
